### Vanilla CTR Zeroshot

In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re

In [9]:
inference_path = 'ctr_vanilla_test_inference_mixtral.pkl'
with open(inference_path, 'rb') as f:
    inference_dict = pickle.load(f)
print(len(inference_dict))

6040


In [4]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [5]:
user_valid_movie_rating_df = ratings_df.groupby('user_id').nth(-2)
user_valid_movie_rating_df.head()

,user_id,movie_id,rating
51,1,1566,4
452,10,2693,5
528,100,1208,2
612,1000,3363,5
989,1001,3756,2


In [6]:
user_test_movie_rating_df = ratings_df.groupby('user_id').nth(-1)
user_test_movie_rating_df.head()

,user_id,movie_id,rating
52,1,48,5
453,10,2252,5
529,100,265,3
613,1000,1104,5
990,1001,647,3


In [10]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 0
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 5584 -- No: 121 -- Invalid: 335
invalid_users: [1039, 1070, 1071, 1091, 1123, 113, 1157, 1226, 1236, 1249, 1250, 1252, 1282, 1284, 1307, 1309, 1310, 1318, 1336, 1363, 1377, 1406, 1412, 1416, 1417, 1431, 1439, 1444, 1453, 1456, 1471, 1488, 1493, 1520, 1534, 1539, 1549, 1582, 1596, 1598, 1604, 1612, 1615, 1621, 1652, 168, 1686, 1702, 171, 1713, 1719, 172, 1751, 1754, 1765, 1766, 1767, 1771, 1801, 1818, 1864, 1892, 1893, 1898, 1907, 191, 1927, 1964, 1993, 2028, 2044, 2045, 2051, 2053, 2056, 207, 209, 2128, 2133, 2160, 217, 2170, 2207, 2210, 2224, 223, 2292, 2295, 2311, 2336, 234, 2347, 2356, 2357, 2369, 2373, 2375, 2379, 2385, 2399, 2431, 2434, 2488, 2490, 250, 2502, 2511, 2516, 2527, 2530, 254, 2542, 2549, 2582, 2601, 2640, 2646, 2653, 2671, 2673, 268, 2686, 2696, 2705, 2707, 2710, 2714, 2725, 2755, 277, 2816, 2819, 2834, 2867, 2871, 2879, 2884, 2887, 2920, 2923, 2925, 2977, 2982, 3040, 3076, 311, 3124, 3133, 3176, 3181, 3222, 3225, 3234, 3236, 3254, 3273, 3282, 3287, 3288, 331, 340

In [11]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_test_movie_rating_df[user_test_movie_rating_df['user_id'] == user]['rating'].values[0]
    # print(original_rating)
    if original_rating >= 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[ 122  944]
 [ 334 4640]]
AUC: 0.523648676683482
Precision: 0.830945558739255
Recall: 0.9328508242862887
Accuracy: 0.7884105960264901


In [31]:
movies_df = pd.read_csv('../movie_summary_25_words.csv')
movies_df[movies_df.movie_id == 1466]

,movie_id,name,cleaned_genre,year,summary
1439,1466,Donnie Brasco,"Crime,Drama",1997,"""Donnie Brasco"" is a gripping crime drama base..."


In [9]:
ctr_valid_dataset_path = './ctr_zeroshot_dataset/ctr_valid_zeroshot_dataset.pkl'
with open(ctr_valid_dataset_path, 'rb') as f:
    ctr_valid_data_dict = pickle.load(f)
print(len(ctr_valid_data_dict))

6040


In [14]:
print(ctr_valid_data_dict[1005])


    You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
    User Profile -
    The user is a male lawyer between 35 to 44 years of age
            The user has a diverse taste in movies, with a preference for those that are thought-provoking and emotionally impactful. They enjoy movies that explore complex themes and blur the lines between reality and fantasy, as seen in their appreciation for "Being John Malkovich" and "Bowfinger." The user also has a soft spot for classic Christmas films, as evidenced by their fondness for "Miracle on 34th Street."
            
            In terms of genres, the user leans towards drama and comedy, with a particular interest in coming-of-age stories and character-driven narratives. They are drawn to movies that feature strong performances and explore the complexities of human relationships, as seen in their appreciation for "Diner" and "Schi

In [14]:
for user in invalid_users:
    print(user, inference_dict[user])

1031 
    Explanation:
    Based on the user's preference for science fiction movies, Gabbeh, an Iranian film, is unlikely to
1039 

To predict whether the user will like or dislike the next movie, we need to analyze the user's movie watching pattern and their preferences. Based
1046 
































1070 
































1091 
































1123 
































1126 
































1154 
    Explanation:
    Based on the user's preference for coming-of-age stories and her dislike for romantic dramas, it
1157 

Answer: Yes

Explanation: The user is a male executive/managerial between 45 to 49 years of age
1171 
































1189 
































1213 

To predict whether the user will like or dislike the next movie, we can analyze the user's movie preferences based on the given data. Here
1236 

Based on the user's viewing history, it appears that she enjoys action-pac